# Implementing Shor's Algorithm using the Qrisp framework on Cat Qubits

This tutorial will walk you through the implementation of the famous Shor's algorithm for prime factoring using Qrisp on the Alice&Bob's Qiskit Provider.

### Why Shor's prime factoring algorithm?

A fair question is to ask why it is of any interest to implement Shor's algorithm and run it with a noise model. It all boils down to the quantum advantage that is provided by the quantum algorithm: the current state-of-the-art classical algorithm for prime factoring is the [general number field sieve](https://en.wikipedia.org/wiki/General_number_field_sieve) (GNFS) which, under some assumptions, achieves the prime factoring of a integer in super-polynomial but sub-exponential time. On the contrary, Shor's algorithm promises a polynomial time to achieve the same task. Equivalently said, Shor's algorithm assures a superpolynomial speedup when compared with a classical algorithm.
Published in 1994 by Peter Shor, Shor's algorithm shook the field of theoretical computer science, giving a glance of how powerful quantum computation could be. The implications of successfully running the Shor's algorithm on an actual machine are several, but let's just cite the most hype-generating one: the security of the popular RSA cryptosystem is based on the assumption that prime factorization is computationally hard. Well, that's no longer true with Shor's algorithm.

### Why Cat Qubits?

However, we are far from running the algorithm efficiently on a quantum computer, since qubits and operations between them are noisy; at the moment, the information stored in the real hardware is completaly lost after few operations. People in the field are frenatically trying to solve this problem by designing new and more efficient [quantum error correction codes](https://en.wikipedia.org/wiki/Quantum_error_correction) that are supposed to detect and correct errors. In the quantum realm, the errors on a elementary piece of information are bidimensional: you can have bit-flips ( $|0\rangle \leftrightarrow |1\rangle$ ) and phase-flips ( $|+\rangle \leftrightarrow |-\rangle$ ). Luckily, there exists a technology that is able to take care of one source of error, by exponentially subpressing the bit-flip rate, virtually allowing to exclusively deal with the other type of error: the cat qubits architecture. We have a whole [series of tutorial](https://github.com/Alice-Bob-SW/emulation-examples/blob/main/0%20-%20An%20introduction%20to%20cat%20qubits.ipynb) if you want to know more about it.

Shor's algorithm requires considerably low error rates to give proper results - we will talk about numbers in a following section, when we will do a sketch of the resource estimation of the algorithm - and therefore error correction is needed. So we tought - wait a second, why don't we use cat qubits then, which are the qubits of the future?

### Why Qrisp?

Qrisp is a high-level programming language for creating and compiling quantum algorithms which features numerous characteristics to smooth the workflow when writing and executing a quantum algorithm.
If you want to get familiar with the Qrisp framework, have a look at the dedicated [series of tutorials](https://qrisp.eu/general/tutorial/index.html) available on Qrisp's website.

Qrisp provides compatibility with Qiskit's framework and it is possible to run Qrisp code directly on the Qiskit's backend making use of the `VirtualQiskitBackend`. This allows us to instantiate the emulator using the [Alice & Bob Qiskit Provider]() as backend and run the code on a cat qubits architechture.


In [34]:
from qrisp import *
from qiskit import *
from qrisp.interface import VirtualQiskitBackend
from qiskit_alice_bob_provider.local.provider import AliceBobLocalProvider

provider = AliceBobLocalProvider()
backend = provider.get_backend('EMU:40Q:LOGICAL_TARGET')
vrtl_qasm_sim = VirtualQiskitBackend(backend)

In [35]:
import math
import random
import numpy as np

We would like to perform the trivial task of prime factoring the number $15$. Everybody knows that $5x3 = 15$, but sometimes is better to double-check - or triple-check, as in this case.

Why don't we start by comparing a classical brute-force period finding algorithm with its quantum counterpart and analyse their performances?
The implementation of this version of the Shor's algorithm is inspired by Qrisp'official tutorial.

#### Classical brute-force period finding


In [65]:
def is_prime(n):
    # returns True if n is prime
    if n <= 1:
        return False
    elif n <= 3:
        return True
    elif n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

def find_period_classical(g,N):
    # returns the period of g in Z_N
    e = 1
    while(pow(g, e, N) != 1):
        e = e+1
    return e

def classical_Shor(N):
    # returns the factors of N by using Shor's algorithm
    # with a classical brute-force period finding routine
    if N % 2 == 0:
        return (2, N // 2)
    if (is_prime(N)):
        return (1,N)
    
    while True:
        a = random.randint(2, N - 1)
        if math.gcd(a, N) != 1:
            return math.gcd(a, N), N // math.gcd(a, N)
        
        r = find_period_classical(a, N)
        
        if r % 2 == 0:
            x = pow(a, r // 2, N)
            p = math.gcd(x - 1, N)
            q = math.gcd(x + 1, N)
            if p != 1 and p != N:
                return (p, N // p)
            if q != 1 and q != N:
                return (q, N // q)

In [37]:
# Test the classical Shor's algorithm for N=10000
N=10000
check_list = list()
for i in range(2,N):
    (p,q) = classical_Shor(i)
    check = (p*q == i)
    if(check==False):
        print(i, (p,q))
else:
    print('Test passed!')

Test passed!


In [42]:
classical_Shor(15)

(3, 5)

The factorisation of 15 only requires small fractions of a second to be achieved with the brute-force algorithm.

NOTE: you can use the `%%timeit` magic command to measure the time it takes to run a cell in a Jupyter notebook, although this will increase the actual time needed to execute the cell since multiple runs are performed. 

#### Quantum Period finding


In [5]:
from sympy import continued_fraction_convergents, continued_fraction_iterator, Rational

def get_r_candidates(approx):
    rationals = continued_fraction_convergents(continued_fraction_iterator(Rational(approx)))
    return [rat.q for rat in rationals]

In [7]:
def get_phase_candidates(a, N):
    qg = QuantumModulus(N)
    qg[:] = 1
    qpe_res = QuantumFloat(2*qg.size + 1, exponent = -(2*qg.size + 1))
    h(qpe_res)
    x=a
    for i in range(len(qpe_res)):
        with control(qpe_res[i]):
            qg *= x
            x = (x*x)%N
    QFT(qpe_res, inv = True)
    qiskit_qc = qpe_res.qs.compile().to_qiskit() # compile the circuit to qiskit for technical limitations
                                                 # of the qrisp framework
    cr = ClassicalRegister(len(qpe_res))
    qiskit_qc.add_register(cr)
    qiskit_qc.measure(range(len(qg),len(qpe_res)+len(qg)),range(len(cr)))
    results = execute(qiskit_qc, backend=backend, shots=5).result().get_counts()
    return results

In [8]:
def binary_to_float(binaries):
    # Convert binary string to floating-point decimal
    results = []
    for binary in binaries:
        decimal = 0
        for i, bit in enumerate(binary):
            if bit == '1':
                decimal += 2 ** -(i + 1)
        results.append(decimal)
    return results

In [9]:
def quantum_shor(a,N):
    meas_res = get_phase_candidates(a,N)
    r_candidates = sum([get_r_candidates(approx) for approx in binary_to_float(meas_res.keys())], [])
    for cand in sorted(set(r_candidates)):
        if (a**cand)%N == 1:
            r = cand
            break
    else:
        raise Exception("Please sample again")
    if r % 2:
        raise Exception("Please choose another a")
    p = np.gcd(a**(r//2)+1, N)
    q = int(N/p)
    if(p>q):
        return(q,p)
    return(p,q)

In [10]:
a=2
N=15
# WARNING: This may take a long time to run
quantum_shor(a,N)

(3, 5)

Notice that the parameter `a` is provided manually, instead of it being drawn randomly as in the classical case: this is merely due to optimisation reason, as many runs of the algorithm are needed in order to find the proper `a`. This could take a long time to achieve!

The remarkable aspect of the Qrisp framework is the compactness and simplicity of implementing a fairly complex algorithm, such as Shor's, in a few lines of code as well as the relatively high level of abstraction with respect to gate-level. This task is much more complex to achieve in Qiskit, for example. You can find either a [specific tutorial to factor 15 with an hard-coded circuit](https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/algorithms/shor_algorithm.ipynb); either a [general implementation](https://github.com/Qiskit/qiskit/blob/9c8eb06984c91156eac078f3d2d73b4cf68370b4/qiskit/algorithms/factorizers/shor.py), which has become deprecated.

In both cases, however, you are forced to operate at gate-level and deal with Qiskit's circuit management technicalities.


### But why is it so slow?
In fact, it takes of the order of minutes to factor $15$.

Suppose for the moment to run a noiseless classical simulation of a quantum algorithm. [Gottesman-Knill theorem](https://en.wikipedia.org/wiki/Gottesman%E2%80%93Knill_theorem) states that circuits composed of Clifford gates can be perfectly simulated in polynomial time on a probabilistic classical computer. Shor's algorithm, however, uses non-Clifford gates and, as a result, the simulation is much longer than in the classical case, in particular for small instances.
On top of that, when adding a noise model, the number of internal operations performed by the Qiskit's backend increases noticeably the computational cost, partially inficiating sparsity assumption of matrix operations and storage.

Anyway, Shor's prime factoring algoritms remains to this day the simplest algorithm to provide a theoretical super-polynomial speedup (if executed on an actual quantum computer) and actually observing it a work could give great insight into the realm of quantum computation.


## Resource Estimation

Let's talk numbers now! How many logical qubits and how many operations would you need to actually solve the problem? Well, for the number of qubits it is quite easy: if $N=2^n$ is the number you want to factorize, you need n qubits to encode its binary representation and $2n +1 $ additional qubits to store the results of the operations. So for $N = 15$, the circuit width of 13. For the number of operations, or depth of the circuit, is a whole different story. In fact, for starters, the Qrisp implementation of the quantum period finding sub-routine requires 657 logical gate. Moreover, each of the gate needs to be transpiled into the native universal set of (logical) gates of the cat qubits architecture. This step increases the number of gates from the previous 657 to 33567!

We define the quantum volume V as the product of the width and the depth of the circuit. In this case, V = 436371.

Now, the logical emulator used here adds the noise model to each gate of the circuit. Suppose that the probability that an error occurs is $p$. Then the probability that no error occurs during the execution of the algorithm is $ (1 - p)^V$. For which error rate the algorithm succeds at least $C\%$ of the times ($0 \leq c\leq 1$) ?

$$
(1-p)^V > C\%
$$

$$
V ln(1-p) > ln(C/100)
$$

$$
1- p > exp\left(\frac{ln(C/100)}{V}\right)
$$

$$
p < 1 -  exp\left(\frac{ln(C/100)}{V}\right)
$$


##### Logical backend comparison


In [45]:
from qiskit_alice_bob_provider.processor.logical_cat import _logical_bit_flip_error, _logical_phase_flip_error

def logical_error_rate(distance,average_nb_photons,kappa_1,kappa_2):
    #returns the logical error rate for the given parameters
    return _logical_bit_flip_error(d=distance, nbar=average_nb_photons)+_logical_phase_flip_error(d=distance, nbar=average_nb_photons, k1=kappa_1, k2=kappa_2)

In [46]:
def error_rate_upper_bound(V, success_rate):
    if success_rate <= 0 or success_rate > 1:
        raise Exception('The success rate should be between 0 and 1')
    return 1 - np.exp(np.log(success_rate)/V)

In [48]:
a =2
N = 15
qg = QuantumModulus(N)
qg[:] = 1
qpe_res = QuantumFloat(2*qg.size + 1, exponent = -(2*qg.size + 1))
h(qpe_res)
x=a
for i in range(len(qpe_res)):
    with control(qpe_res[i]):
        qg *= x
        x = (x*x)%N
QFT(qpe_res, inv = True)

<QuantumFloat 'qpe_res'>

In [49]:
# Parameters of the `EMU:40Q:LOGICAL_TARGET` backend
distance = 15
n_qubits = 40 
kappa_1 = 100
kappa_2= 1e7
average_nb_photons=19

In [50]:
qc_qiskit_TARGET = qpe_res.qs.compile().to_qiskit()

In [51]:
print(qc_qiskit_TARGET.width())
print(qc_qiskit_TARGET.depth())

13
654


In [52]:
qc_qiskit_transpiled_TARGET = transpile(qc_qiskit_TARGET,backend=backend) 

In [53]:
print(qc_qiskit_transpiled_TARGET.width())
print(qc_qiskit_transpiled_TARGET.depth())

13
33412


In [54]:
q_volume_TARGET = qc_qiskit_transpiled_TARGET.depth() * (qc_qiskit_transpiled_TARGET.width())  #width * depth of the circuit
print(q_volume_TARGET)

434356


In [55]:
print(error_rate_upper_bound(q_volume_TARGET, 0.5))
print(logical_error_rate(distance,average_nb_photons,kappa_1,kappa_2))


1.5958030451512784e-06
6.656827183107874e-15


The error rate of the backend is way lower than the upper bound, so the algorithm should succed with high probability. 

Let's now use a noisier backend.

In [56]:
backend = provider.get_backend('EMU:15Q:LOGICAL_EARLY')

In [57]:
# Parameters of the `EMU:15Q:LOGICAL_EARLY` backend
distance = 13
n_qubits = 15 
kappa_1 = 100
kappa_2= 100000
average_nb_photons=7

In [58]:
qc_qiskit_EARLY = qpe_res.qs.compile().to_qiskit()

In [59]:
print(qc_qiskit_EARLY.width())
print(qc_qiskit_EARLY.depth())

13
655


In [60]:
qc_qiskit_transpiled_EARLY = transpile(qc_qiskit_EARLY,backend=backend) 

In [61]:
print(qc_qiskit_transpiled_EARLY.width())
print(qc_qiskit_transpiled_EARLY.depth())

13
33409


In [62]:
q_volume_EARLY = qc_qiskit_transpiled_EARLY.depth() * (qc_qiskit_transpiled_EARLY.width())  #width * depth of the circuit
print(q_volume_EARLY)

434317


In [63]:
print(error_rate_upper_bound(q_volume_TARGET, 0.5))
print(logical_error_rate(distance,average_nb_photons,kappa_1,kappa_2))

1.5958030451512784e-06
0.0015488351907778613


In this case, the error rate of the backend is larger than the upper bound, which means that the algorithm is not expected to succeed at each run.

In [64]:
# WARNING: This may take a long(long) time to run
for _ in range(10):
     print(quantum_shor(a,N))

(3, 5)
(1, 15)
(3, 5)
(3, 5)
(3, 5)
(3, 5)
(3, 5)
(3, 5)
(3, 5)
(3, 5)


That's about it folks! If you want to know more about this ground-breaking algorithm, hava a glance at the following section.

## Further readings:

- [Shor's original paper](https://arxiv.org/abs/quant-ph/9508027)
- [Resource estimation for cracking the 2048-bit RSA encryption](https://arxiv.org/abs/2103.06159)
- [Scott Aaronson's blog post on the intuition behind Shor's algorithm](https://scottaaronson.blog/?p=208)
